In [40]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

True

### Understanding Stores in LangChain

In [64]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_path = r"data\montreal.pdf"

loader = PyPDFLoader(file_path=file_path)

# by default, we will split by pages with no text_splitter
documents = loader.load_and_split(text_splitter=None)
documents

[Document(page_content="Things to Do in Montreal \nMontreal is a vibrant city with a rich cultural heritage and an array of activities to suit every \ninterest. \nOld Montreal  \nStart your exploration in the historic Old Montreal, where cobblestone streets and 17th-\ncentury architecture transport you back in time. Visit the stunning Notre-Dame Basilica, \nrenowned for its intricate interior and dramatic light shows. \nMontreal Museum of Fine Arts  \nFor a taste of the local arts scene, head to the Montreal Museum of Fine Arts, home to an \nimpressive collection of Canadian and international works. \nMont Royal  \nIf you're an outdoor enthusiast, Mont Royal o Ưers scenic hiking trails and panoramic views \nof the city. In the summer, the park becomes a hub for picnics and outdoor events, \nincluding the popular Tam-Tams festival, where locals gather to enjoy music and dance. \nMile End District   \nFor a modern twist, the Mile End district is a must-visit. Known for its bohemian vibe,

### Unsing the PostgresByteStore

In [58]:
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_postgres import PGVector
from database import COLLECTION_NAME, CONNECTION_STRING
from utils.store import PostgresByteStore
from langchain_postgres import PostgresSaver, PickleCheckpointSerializer
from langchain.indexes import SQLRecordManager, index

embeddings = OpenAIEmbeddings()
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=CONNECTION_STRING,
    use_jsonb=True,
)

store = PostgresByteStore(CONNECTION_STRING, COLLECTION_NAME)
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore, 
    docstore=store, 
    id_key=id_key,
)

# define record manager
namespace = f"pgvector/{COLLECTION_NAME}"
record_manager = SQLRecordManager(
    namespace, db_url=CONNECTION_STRING
)
record_manager.create_schema()

retriever

MultiVectorRetriever(vectorstore=<langchain_postgres.vectorstores.PGVector object at 0x000002E90AACA450>, docstore=<utils.store.PostgresByteStore object at 0x000002E90AAF3E90>)

In [65]:
import uuid

# Add a unique doc_id to each document's metadata
for doc in documents:
    doc.metadata["doc_id"] = str(uuid.uuid4())

In [66]:
documents

[Document(page_content="Things to Do in Montreal \nMontreal is a vibrant city with a rich cultural heritage and an array of activities to suit every \ninterest. \nOld Montreal  \nStart your exploration in the historic Old Montreal, where cobblestone streets and 17th-\ncentury architecture transport you back in time. Visit the stunning Notre-Dame Basilica, \nrenowned for its intricate interior and dramatic light shows. \nMontreal Museum of Fine Arts  \nFor a taste of the local arts scene, head to the Montreal Museum of Fine Arts, home to an \nimpressive collection of Canadian and international works. \nMont Royal  \nIf you're an outdoor enthusiast, Mont Royal o Ưers scenic hiking trails and panoramic views \nof the city. In the summer, the park becomes a hub for picnics and outdoor events, \nincluding the popular Tam-Tams festival, where locals gather to enjoy music and dance. \nMile End District   \nFor a modern twist, the Mile End district is a must-visit. Known for its bohemian vibe,

In [67]:
from langchain.schema.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Assume documents is a list of Document objects with 'doc_id' in their metadata
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

all_sub_docs = []
for doc in documents:
    doc_id = doc.metadata.get("doc_id")  # Retrieve the doc_id from the document's metadata
    sub_docs = child_text_splitter.split_documents([doc])
    for sub_doc in sub_docs:
        sub_doc.metadata["doc_id"] = doc_id  # Assign the same doc_id to each sub-document
    all_sub_docs.extend(sub_docs)

# Display the resulting sub-documents
all_sub_docs


[Document(page_content='Things to Do in Montreal \nMontreal is a vibrant city with a rich cultural heritage and an array of activities to suit every \ninterest. \nOld Montreal  \nStart your exploration in the historic Old Montreal, where cobblestone streets and 17th-\ncentury architecture transport you back in time. Visit the stunning Notre-Dame Basilica, \nrenowned for its intricate interior and dramatic light shows.', metadata={'source': 'data\\montreal.pdf', 'page': 0, 'doc_id': 'df373aa6-8ee1-42e4-a8a5-c1b1d1b60490'}),
 Document(page_content='century architecture transport you back in time. Visit the stunning Notre-Dame Basilica, \nrenowned for its intricate interior and dramatic light shows. \nMontreal Museum of Fine Arts  \nFor a taste of the local arts scene, head to the Montreal Museum of Fine Arts, home to an \nimpressive collection of Canadian and international works. \nMont Royal', metadata={'source': 'data\\montreal.pdf', 'page': 0, 'doc_id': 'df373aa6-8ee1-42e4-a8a5-c1b1d1

In [73]:
idx = index(all_sub_docs, record_manager, vectorstore, cleanup="incremental",
                                          source_id_key="source")
# Generate the list of (doc_id, document) tuples from the documents
doc_id_document_tuples = [(doc.metadata["doc_id"], doc) for doc in documents]

# Pass the list of tuples to retriever.docstore.mset
retriever.docstore.mset(doc_id_document_tuples)

In [75]:
idx

### Creating Summaries for Each Parent Chunk

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt_text = """You are an assistant tasked with summarizing text. \
Directly summarize the following text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Initialize the Language Model (LLM)
model = ChatOpenAI(temperature=0, model="gpt-4o")

# Define the summary chain
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
parent_chunk = [i.page_content for i in documents]
text_summaries = summarize_chain.batch(parent_chunk, {"max_concurrency": 5})

In [ ]:
# from langchain.schema.document import Document

# # Extract metadata from documents
# metadata = [i.metadata for i in documents]

# # Create new Document objects with the summaries and the original metadata
# text_summaries_with_metadata = [
#     Document(page_content=summary, metadata=meta)
#     for summary, meta in zip(text_summaries, metadata)
# ]

In [ ]:
from langchain.schema.document import Document

# Extract metadata from documents
metadata = [i.metadata for i in documents]

# Create new Document objects with the summaries and the original metadata
text_summaries_with_metadata = []
for summary, meta in zip(text_summaries, metadata):
    # Add the suffix "(summary)" to the source in metadata
    if 'source' in meta:
        meta['source'] += ' (summary)'
    text_summaries_with_metadata.append(Document(page_content=summary, metadata=meta))


In [ ]:
text_summaries_with_metadata

In [ ]:
idx = index(text_summaries_with_metadata, record_manager, vectorstore, cleanup="incremental",
                                          source_id_key="source")
# Generate the list of (doc_id, document) tuples from the documents
doc_id_document_tuples = [(doc.metadata["doc_id"], doc) for doc in documents]

# Pass the list of tuples to retriever.docstore.mset
retriever.docstore.mset(doc_id_document_tuples)

In [ ]:
idx

In [ ]:
# retriever.vectorstore.add_documents(summary_docs)
# retriever.docstore.mset(list(zip(doc_ids, documents)))

### Generating Hypothetical Questions for Each Parent Chunk

In [ ]:
functions = [
    {
        "name": "hypothetical_questions",
        "description": "Generate hypothetical questions",
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["questions"],
        },
    }
]

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

question_chain = (
    {"doc": lambda x: x.page_content}
    # Only asking for 5 hypothetical questions, but this could be adjusted
    | ChatPromptTemplate.from_template(
        """Generate a list of exactly 5 hypothetical questions that the below document could be used to answer:\n\n{doc}
        seperate each question with a comma (,)
        """
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

In [ ]:
hypothetical_questions = question_chain.batch(documents, {"max_concurrency": 5})

In [ ]:
hypothetical_questions

In [ ]:
from langchain.schema.document import Document

hypothetical_docs = []
for question_list, doc_id in zip(hypothetical_questions, doc_ids):
    for question in question_list:
        # Define your new metadata here
        new_metadata = {"doc_id": doc_id}

        # Create a new Document instance for each question
        # The question itself is the page_content
        doc = Document(page_content=question, metadata=new_metadata)

        # Add the Document to the list
        hypothetical_docs.append(doc)

In [ ]:
hypothetical_docs

In [ ]:
idx = index(hypothetical_docs, record_manager, vectorstore, cleanup="incremental",
                                          source_id_key="source")
retriever.docstore.mset(list(zip(doc_ids, documents)))

In [ ]:
idx

In [ ]:
# retriever.vectorstore.add_documents(hypothetical_docs)
# retriever.docstore.mset(list(zip(doc_ids, documents)))

In [ ]:
retriever.vectorstore.similarity_search("What dining options are available in Montreal for those interested in Middle Eastern cuisine?")

In [ ]:
retriever.invoke("What dining options are available in Montreal for those interested in Middle Eastern cuisine?")

### Creating an LCEL Chain and Testing the Retriever

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt template
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("What dining options are available in Montreal for those interested in Middle Eastern cuisine?")

In [ ]:
chain.invoke("Where can I find the best smoked meat sandwiches in Montreal?")

In [ ]:
chain.invoke("Where can I find the best food in Montreal?")